In [ ]:
import pybaseball as pb
import pandas as pd
import altair as alt

In [ ]:
def get_bwar(year=2021):
    df = pb.bwar_bat().query('year_ID == 2021')
    df = df.groupby(["player_ID", "year_ID", "name_common"])["WAR"].sum().reset_index()
    df.columns = ["IDbref", "Season", "Name", "bWAR"]
    return df
    

def get_fwar(year=2021):
    df = pb.batting_stats(year, qual = 0)
    df = df.rename({"WAR" : "fWAR"}, axis=1)
    
    return df[["IDfg", "fWAR"]]

In [ ]:
get_bwar()

In [ ]:
get_bwar()

In [ ]:
fwar = get_fwar()

In [ ]:
id_df = pb.playerid_reverse_lookup(fwar.IDfg, key_type="fangraphs")[["key_fangraphs", "key_bbref"]]

In [ ]:
fwar_with_ids = fwar.merge(id_df,
           left_on="IDfg",
           right_on="key_fangraphs")

In [ ]:
fwar_bwar = fwar_with_ids.merge(
    get_bwar(),
    left_on = "key_bbref",
    right_on = "IDbref"
)

In [ ]:
fwar_bwar["diff"] = fwar_bwar.fWAR - fwar_bwar.bWAR

In [ ]:
alt.Chart(fwar_bwar).mark_circle().encode(
    alt.X("fWAR"),
    alt.Y("bWAR"),
    alt.Tooltip(["Name", "fWAR", "bWAR", "diff"])
).properties(
    height = 400,
    width = 600
).interactive()

In [ ]:
fwar_bwar.to_csv("bwar_v_fwar.csv", index=False)